### Relax Inc. mock takehome challenge
Task: Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate
days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user
adoption .

In [28]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from datetime import datetime

#### Read files

In [2]:
engagement_file = "takehome_user_engagement.csv"
users_file = "takehome_users2.csv"

In [50]:
engagement = pd.DataFrame(pd.read_csv(engagement_file, parse_dates=True))
users = pd.DataFrame(pd.read_csv(users_file, parse_dates=True, encoding = 'latin1'))

In [160]:
print('the total number of users is',len(users))
users[:3]

the total number of users is 12000


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0


In [32]:
engagement[:3]

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1


In [51]:
engagement['time_stamp'] = engagement.time_stamp.apply(lambda d: datetime.strptime(d, '%Y-%m-%d %H:%M:%S'))

In [52]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [146]:
engagement['date']= engagement['time_stamp'].apply(lambda s: s.date())
engagement.drop(['visited','time_stamp'], axis=1, inplace=True)

In [147]:
engagement.head()

,user_id,date
0,1,2014-04-22
1,2,2013-11-15
2,2,2013-11-29
3,2,2013-12-09
4,2,2013-12-25


#### Adopted users
Identify which users have logged into the product on 3 days in one 7-day period.
<br>
use engagement table -- ```user_id``` == ```object_id```

Strategy: 
- count visits per day by user
- check there aren't multiple visits/day
- use timedelta to calculate difference between visits, and check if <= 7

In [150]:
from datetime import datetime, timedelta

def adopted(a):
    if len(a) >= 3:  
        a = [i for i in a]  # convert property object to list
        a.sort()
        a = [a[i+1] - a[i] for i in range(len(a)-2)]  # cumulative difference of current and next day
        a = [1 for i in range(len(a)-2) if a[i] + a[i+1] + a[i+2] <= timedelta(days=7)]
        if 1 in a:
            return 1

This fucntion converts an object to list, then returns 1 if a user logged into the product on three separate days in at least one seven day period 

In [152]:
df = engagement.groupby('user_id').agg(adopted)  # group by user_id and aggregate using custom function
df.fillna(0, inplace=True)  # fill null values with 0
df.columns = ['adopted_user']
df.head()

,adopted_user
user_id,
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0


In [163]:
print('The number of adopted users is', len(df[df['adopted_user']==1]),'of 12000 total.')

The number of adopted users is 1322 of 12000 total.
